In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/LIDCI/5/PROCESAMIENTO DE LENGUAJE NATURAL/multilang_reviews.csv')
df.head()

,Review
0,The Canon EOS R5 exceeded my expectations. The...
1,Die Sony Alpha 7 III ist eine hervorragende Ka...
2,Le Nikon Z6 II est incroyable pour la vidéo. L...
3,La Fujifilm X-T4 tiene un rendimiento excelent...
4,我非常喜歡我的 Sony A6400，自動對焦系統非常出色，適合拍攝活動中的孩子。


In [ ]:
df =df.head(10)

In [ ]:
api_key="enter_api_key_here"

In [ ]:
#!pip install langchain-google-genai para usar gemini
!pip install langchain_openai
!pip install OpenAi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [ ]:
from re import S
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import Markdown, display
import json
import re


llm = ChatOpenAI(
    model= "gpt-3.5-turbo", #"gemini-1.5-pro"
    openai_api_key=api_key
)

SYSTEM_PROMPT = """
IMPORTANTE: JAMÁS incluyas ```json al inicio o al final. Debo convertir la respuesta a un diccionario en python usando json.loas()

Eres un experto en revisar reseñas de productos dejadas por clientes.
Para la siguiente reseña, quiero que extraigas lo siguiente:
0) La reseña original sin cambios
1) El sentimiento (Positivo, Negativo o Neutro)
2) El idioma en el que la reseña está escrita
3) La explicación de porque la reseña fue calificada como positiva, negativa o neutra
4) La traducción de la reseña analizada a español
Tu respuesta debe seguir exclusivamente este formato
{
  'REVIEW': La reseña original sin cambio alguno
  'SENTIMIENTO': El sentimiento detectado,
  'IDIOMA': El idioma detectado,
  'EXPLICACIÓN': La explicación del por que de la calificación de la reseña,
  'TRADUCCIÓN': La traducción de la reseña

}
Responde UNICAMENTE CON ESE JSON. No devuelvas nada más.
ASEGURATE: que jamas incluyas ```json al inicio o al final. Debo convertir la respuesta a un diccionario en python usando json.loas()
"""
PROMPT = "La CANON PRO 2502 es la mejor camara del universo"

messages = [
    SystemMessage(content=SYSTEM_PROMPT),
    HumanMessage(content=PROMPT)
]
respuesta = llm.invoke(messages)
respuesta.content

"{\n  'REVIEW': 'La CANON PRO 2502 es la mejor camara del universo',\n  'SENTIMIENTO': 'Positivo',\n  'IDIOMA': 'Español',\n  'EXPLICACIÓN': 'La reseña fue calificada como positiva porque el cliente expresa de forma entusiasta que la cámara CANON PRO 2502 es la mejor del universo.',\n  'TRADUCCIÓN': 'La CANON PRO 2502 es la mejor cámara del universo'\n}"

In [ ]:
import json
import re

# The previous attempt failed because the JSON string used single quotes instead of double quotes for keys.
# Also, sometimes the response might include ```json and ``` delimiters.
# We will clean the string before loading it as JSON.

respuesta_content = respuesta.content.strip()
# Remove leading/trailing ```json and ``` if present
if respuesta_content.startswith('```json'):
    respuesta_content = respuesta_content[7:].strip()
if respuesta_content.endswith('```'):
    respuesta_content = respuesta_content[:-3].strip()

# Replace single quotes around keys with double quotes using a more robust approach
# This replaces single quotes only when they are around keys (alphanumeric followed by a colon)
respuesta_content = re.sub(r"'(?P<key>[a-zA-Z0-9_]+)'\s*:", r'"\g<key>":', respuesta_content)


try:
    respuesta_diccionario = json.loads(respuesta_content)
    print("JSON parsed successfully:")
    print(respuesta_diccionario)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON:")
    print(f"JSON string was: {respuesta_content}")
    print(f"Error: {e}")
    respuesta_diccionario = None # Or handle the error appropriately

Error decoding JSON:
JSON string was: {
  "REVIEW": 'La CANON PRO 2502 es la mejor camara del universo',
  "SENTIMIENTO": 'Positivo',
  "IDIOMA": 'Español',
  'EXPLICACIÓN': 'La reseña fue calificada como positiva porque el cliente expresa de forma entusiasta que la cámara CANON PRO 2502 es la mejor del universo.',
  'TRADUCCIÓN': 'La CANON PRO 2502 es la mejor cámara del universo'
}
Error: Expecting value: line 2 column 13 (char 14)


In [ ]:
type(respuesta_diccionario)

NoneType

In [ ]:
from re import S
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import Markdown, display
import json
import re


llm = ChatOpenAI(
    model= "gpt-3.5-turbo", #"gemini-1.5-pro"
    openai_api_key=api_key
)

SYSTEM_PROMPT = """
IMPORTANTE: JAMÁS incluyas ```json al inicio o al final. Debo convertir la respuesta a un diccionario en python usando json.loas()

Eres un experto en revisar reseñas de productos dejadas por clientes.
Para la siguiente reseña, quiero que extraigas lo siguiente:
0) La reseña original sin cambios
1) El sentimiento (Positivo, Negativo o Neutro)
2) El idioma en el que la reseña está escrita
3) La explicación de porque la reseña fue calificada como positiva, negativa o neutra
4) La traducción de la reseña analizada a español
Tu respuesta debe seguir exclusivamente este formato
{
  'REVIEW': La reseña original sin cambio alguno
  'SENTIMIENTO': El sentimiento detectado,
  'IDIOMA': El idioma detectado,
  'EXPLICACIÓN': La explicación del por que de la calificación de la reseña,
  'TRADUCCIÓN': La traducción de la reseña

}
Responde UNICAMENTE CON ESE JSON. No devuelvas nada más.
ASEGURATE: que jamas incluyas ```json al inicio o al final. Debo convertir la respuesta a un diccionario en python usando json.loas()
"""

def llm_resenias(review):
  messages = [
      SystemMessage(content=SYSTEM_PROMPT),
      HumanMessage(content=review)
  ]
  respuesta = llm.invoke(messages)
  respuesta_content = respuesta.content.strip()

  # Remove leading/trailing ```json and ``` if present
  if respuesta_content.startswith('```json'):
      respuesta_content = respuesta_content[7:].strip()
  if respuesta_content.endswith('```'):
      respuesta_content = respuesta_content[:-3].strip()

  # Replace single quotes around keys with double quotes
  # This is a simple regex that assumes keys are alphanumeric and followed by a colon
  respuesta_content = re.sub(r"'([^']*)':", r'"\1":', respuesta_content)

  try:
    respuesta = json.loads(respuesta_content)
  except json.JSONDecodeError as e:
    print(f"Error decoding JSON for review: {review}")
    print(f"JSON string was: {respuesta_content}")
    print(f"Error: {e}")
    respuesta = None # Or handle the error appropriately
  return respuesta

In [ ]:
respuestas = []
for indice, fila in df.iterrows():
  print(f"Procesando reseña {indice+1} de {len(df)}")
  respuestas.append(llm_resenias(fila['Review']))


Procesando reseña 1 de 10
Error decoding JSON for review: The Canon EOS R5 exceeded my expectations. The 45MP sensor captures incredible detail and the autofocus is lightning fast.
JSON string was: {
  "REVIEW": 'The Canon EOS R5 exceeded my expectations. The 45MP sensor captures incredible detail and the autofocus is lightning fast.',
  "SENTIMIENTO": 'Positivo',
  "IDIOMA": 'Inglés',
  "EXPLICACIÓN": 'La reseña fue calificada como positiva porque el cliente menciona que la cámara superó sus expectativas, el sensor de 45MP captura detalles increíbles y el enfoque automático es muy rápido.',
  "TRADUCCIÓN": 'La Canon EOS R5 superó mis expectativas. El sensor de 45MP captura detalles increíbles y el enfoque automático es muy rápido.'
}
Error: Expecting value: line 2 column 13 (char 14)
Procesando reseña 2 de 10
Error decoding JSON for review: Die Sony Alpha 7 III ist eine hervorragende Kamera. Die Bildqualität bei schlechten Lichtverhältnissen ist beeindruckend.
JSON string was: {
  "RE

In [14]:
final = pd.DataFrame(respuestas)
final

,0
0,None
1,None
2,None
3,None
4,None
5,None
6,None
7,None
8,None
9,None
